# Extract Info from postisrccd, calexp and calexpbg in Selected visits LSST in Auxtel

- author Sylvie Dagoret-Campagne
- creation date 2024-05-07
- last update 2024-05-09
- affiliation : IJCLab
- kernel : **w_2024_16**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS


import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

In [2]:
import gc

In [3]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [4]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [5]:
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay
import lsst.afw.table as afwTable
import lsst.geom as geom

In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

Import statements that we will need later

Let's make a new plot and metric tool, we'll base it on the example in the getting started guide.

In [8]:
# INSERT YOUR collection and tract
# for rehearsal use collection 2 which have CCDvisit
butlerRoot = "/repo/embargo"

collection1 = 'LSSTComCamSim/runs/nightlyvalidation/20240402/d_2024_03_29/DM-43612'
collection2 = 'LSSTComCamSim/runs/nightlyvalidation/20240403/d_2024_03_29/DM-43612'
collection3 = 'LSSTComCamSim/runs/nightlyvalidation/20240404/d_2024_03_29/DM-43612'
collection = 'LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985'
collectionn = collection
#collections = [collection1,collection2,collection3]
collections = [collection]
collectionStr = collectionn.replace("/", "_")
fn_ccdVisit_tracts_patches = f"ccdVisittractpatch_{collectionStr}.csv"
instrument = 'LATISS'
skymapName = "latiss_v1"
where_clause = "instrument = \'" + instrument+ "\'"
tract = 3864 # mostly for light-curves
patch_sel = 236
band = 'g'
#tract = 5615
# tract = 5634 # interesting to view calib parameters
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"

In [9]:
#dataId = {"skymap": "latiss_v1", "tract": 5615, "instrument": "LATISS"}
dataId = {"skymap": skymapName, "tract": tract, "instrument": instrument}
repo = '/sdf/group/rubin/repo/oga/'
butler = dafButler.Butler(repo)
#t = Butler.get(table_sel, dataId=dataId, collections=collections)
registry = butler.registry

In [10]:
skymap = butler.get('skyMap', skymap=skymapName, collections=collections)

In [11]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [12]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [13]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

## Get pixel scale

In [14]:
import lsst.geom as geom
import lsst.sphgeom

skymap = butler.get('skyMap', skymap=skymapName, collections=collections )
tractInfo = skymap.generateTract(tract)
for patch in tractInfo:    
    patchID = patch.getSequentialIndex()
        
    ibb=patch.getInnerBBox()
    tWCS=tractInfo.getWcs()
       
    # loop on the 4 corners
    for icorn,corner in enumerate(ibb.getCorners()):
        p = geom.Point2D(corner.getX(), corner.getY())
        coord = tWCS.pixelToSky(p)


In [15]:
tWCS

FITS standard SkyWcs:
Sky Origin: (122.4365482234, -36.4462809917)
Pixel Origin: (28499, 28499)
Pixel Scale: 0.1 arcsec/pixel

In [16]:
#arcsec/pixel
pixel_scale = tWCS.getPixelScale().asArcseconds()

## Selected visits

In [17]:

# get the csv file produced by stat_on_visits_LSSTAuxtel.ipynb in ../Visits
#file_selected_visits = "../Visits/ccdVisittractpatch_LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240311_w_2024_10_PREOPS-4985.csv"
file_selected_visits = os.path.join("../Visits",fn_ccdVisit_tracts_patches)
df_myselectedvisits = pd.read_csv(file_selected_visits,index_col=0)
if 0:

    # select the band
    cut = (df_myselectedvisits.band == band) &  (df_myselectedvisits.tractID == tract) & (df_myselectedvisits.patchID == patch_sel)
    df_myselectedvisits = df_myselectedvisits[cut]

#move the visitid as a column not to loose it during the merge 
df_myselectedvisits.reset_index(inplace=True) 


In [18]:
index = 0
visitId = df_myselectedvisits.iloc[index]['visitId']
band = df_myselectedvisits.iloc[index]['band']
det = df_myselectedvisits.iloc[index]['detector']
dataId = {'visit': visitId, 'instrument':instrument , 'detector': det}

Note on the CCD visit here:
- zeroPoint is in ABMag relative to a flux of 1ADU/sec
- skyBg is in ADU , ADU/s , nJ per pixel ?
- skyNoise is in ADU, ADU/s , nJ  per pixel ?

In [19]:
df_myselectedvisits.iloc[index]

ccdVisitId      40919696896
visitId       2023051100262
band                      r
detector                  0
ra               239.946728
dec              -24.053747
llcra            239.935567
llcdec           -24.128923
ulcra            240.029265
ulcdec           -24.062575
urcra            239.957875
urcdec            -23.97857
lrcra            239.864202
lrcdec           -24.044874
ccdVid                    0
Vid                       0
nightObs           20230511
tractID                5615
patchID                 295
zeroPoint         27.697306
airmass            1.823818
skyBg             11.193549
skyNoise             7.8568
expTime                30.0
Name: 0, dtype: object

## One postISRCCD and One Calexp

In [20]:
def ComputeStatfromImage(arr,remove_zero_flag=False):
    data_flat = arr.flatten()

    if remove_zero_flag:
        data_flat_nozero = data_flat[~(data_flat==0)]
        data_flat = data_flat_nozero 
        

    mu = np.mean(data_flat)
    med = np.median(data_flat)
    std = np.std(data_flat)
    sigMad = 1.4826 * np.median(np.fabs(data_flat - med))
    vmin,vmax = med - 3 * sigMad, med + 3 * sigMad
    return mu,med,std,sigMad,vmin,vmax


In [21]:
calexp = butler.get('calexp', **dataId,collections=collections)
bkgd  = butler.get('calexpBackground', **dataId,collections=collections)
# get the calibration constant  : expo_photocalibconstant_mean in nJ per ADU. To convert it in AB-Mag : convert_fluxtomag(expo_photocalibconstant_mean) --> zero point
calexp_info = calexp.getInfo()
photocalib = calexp_info.getPhotoCalib()
expo_photocalibconstant_mean = photocalib.getCalibrationMean()
expo_photocalibconstant_error = photocalib.getCalibrationErr()
calexp_md = calexp_info.getMetadata()
magzero,magzero_rms,magzero_nobj = calexp_md["MAGZERO"],calexp_md["MAGZERO_RMS"],calexp_md["MAGZERO_NOBJ"] 
bgmean,bgvar = calexp_md["BGMEAN"],calexp_md["BGVAR"]


mu_ce,med_ce,std_ce,sigMad_ce,vmin_ce,vmax_ce = ComputeStatfromImage(calexp.getImage().array,remove_zero_flag=False)
mu_bkg,med_bkg,std_bkg,sigMad_bkg,vmin_bkg,vmax_bkg = ComputeStatfromImage(bkgd.getImage().array,remove_zero_flag=False)

datasetRefs = registry.queryDatasets('postISRCCD', dataId=dataId, collections  = collections)
for i, ref in enumerate(datasetRefs):
    postisrccd  = butler.get(ref)
mu_pisr,med_pisr,std_pisr,sigMad_pisr,vmin_pisr,vmax_pisr = ComputeStatfromImage(postisrccd.getImage().array,remove_zero_flag=False)

mask = calexp.mask.array
the_mask = np.where(mask==0,1,0)

mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,vmin_ce_m,vmax_ce_m = ComputeStatfromImage(calexp.getImage().array*the_mask ,remove_zero_flag=True)
mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,vmin_bkg_m,vmax_bkg_m = ComputeStatfromImage(bkgd.getImage().array*the_mask,remove_zero_flag=True)
mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m,vmin_pisr_m,vmax_pisr_m = ComputeStatfromImage(postisrccd.getImage().array*the_mask,remove_zero_flag=True)

del calexp,bkgd,postisrccd,mask,the_mask

In [22]:
print(f"{index}) ======================{visitId} == {band}  == {det} =====================")
print('* photocalib ::')
print(photocalib)
print(expo_photocalibconstant_mean,convert_fluxtomag(expo_photocalibconstant_mean) )
print("magzero,magzero_rms,magzero_nobj :",magzero,magzero_rms,magzero_nobj)
print("bgmean,bgvar :",bgmean,bgvar)

print("* calexp ::")
print(mu_ce,med_ce,std_ce,sigMad_ce,vmin_ce,vmax_ce)
print(mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,vmin_ce_m,vmax_ce_m)
print("* calexpBackground :: ")
print(mu_bkg,med_bkg,std_bkg,sigMad_bkg,vmin_bkg,vmax_bkg)
print(mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,vmin_bkg_m,vmax_bkg_m)
print("* postisrccd :: ")
print(mu_pisr,med_pisr,std_pisr,sigMad_pisr,vmin_pisr,vmax_pisr)
print(mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m,vmin_pisr_m,vmax_pisr_m) 

0) ======================2023051100262 == r  == 0 =====================
* photocalib ::
spatially constant with mean: 30.4905 error: 0.0514512
30.49046409792168 27.689589912878567
magzero,magzero_rms,magzero_nobj : 23.9967867760794 0.00183212641679645 36
bgmean,bgvar : 11.3790014454661 0.0632380885261264
* calexp ::
1.7696625 0.11838882 254.89998 7.989821800231933 -23.851076583981513 24.087854217410086
0.00023732598019618814 0.007767405128106475 7.9552807771891105 7.9355474649099165 -23.798874989601643 23.814409799857856
* calexpBackground :: 
11.279222 11.335263 0.25390127 0.2340415288925171 10.63313866558075 12.037387838935851
11.287554528754608 11.33798885345459 0.24836555978148378 0.2255848880767822 10.661234189224244 12.014743517684936
* postisrccd :: 
13.167816 11.390819 266.133 7.9942487760543814 -12.591927732276915 35.373564924049376
11.287792617046525 11.287638664245605 7.959123604637137 7.940485975456237 -12.533819262123107 35.109096590614314


In [23]:
N = len(df_myselectedvisits)
NMAX = 1000

In [41]:
df = pd.DataFrame(columns=['idx', 'visitId','ccdVisitId','band', 'detector',
                           'zeroPoint','airmass','skyBg','skyNoise','expTime',
                           'photocalib_m','photocalib_e',
                           'magzero','magzero_rms','magzero_nobj',
                           'bgmean','bgvar',
                           'mu_ce','med_ce','std_ce','sigMad_ce',
                           'mu_ce_m','med_ce_m','std_ce_m','sigMad_ce_m',
                           'mu_bkg','med_bkg','std_bkg','sigMad_bkg',
                           'mu_bkg_m','med_bkg_m','std_bkg_m','sigMad_bkg_m',
                           'mu_pisr','med_pisr','std_pisr','sigMad_pisr',
                           'mu_pisr_m','med_pisr_m','std_pisr_m','sigMad_pisr_m'
                          ])

In [38]:
for index in range(N):

    visitId = df_myselectedvisits.iloc[index]['visitId']
    ccdvisitId = df_myselectedvisits.iloc[index]['ccdVisitId']
    band =  df_myselectedvisits.iloc[index]['band']
    det = df_myselectedvisits.iloc[index]['detector']   

    zeroPoint = df_myselectedvisits.iloc[index]['zeroPoint']
    airmass = df_myselectedvisits.iloc[index]['airmass']          
    skyBg  = df_myselectedvisits.iloc[index]['skyBg']
    skyNoise = df_myselectedvisits.iloc[index]['skyNoise']       
    expTime   = df_myselectedvisits.iloc[index]['expTime']

    dataId = {'visit': visitId, 'instrument':instrument , 'detector': det}
    
    
    calexp = butler.get('calexp', **dataId,collections=collections)
    bkgd  = butler.get('calexpBackground', **dataId,collections=collections)
    # get the calibration constant  : expo_photocalibconstant_mean in nJ per ADU. To convert it in AB-Mag : convert_fluxtomag(expo_photocalibconstant_mean) --> zero point
    calexp_info = calexp.getInfo()
    photocalib = calexp_info.getPhotoCalib()
    expo_photocalibconstant_mean = photocalib.getCalibrationMean()
    expo_photocalibconstant_error = photocalib.getCalibrationErr()
    calexp_md = calexp_info.getMetadata()
    magzero,magzero_rms,magzero_nobj = calexp_md["MAGZERO"],calexp_md["MAGZERO_RMS"],calexp_md["MAGZERO_NOBJ"] 
    bgmean,bgvar = calexp_md["BGMEAN"],calexp_md["BGVAR"]

    mu_ce,med_ce,std_ce,sigMad_ce,vmin_ce,vmax_ce = ComputeStatfromImage(calexp.getImage().array,remove_zero_flag=False)
    mu_bkg,med_bkg,std_bkg,sigMad_bkg,vmin_bkg,vmax_bkg = ComputeStatfromImage(bkgd.getImage().array,remove_zero_flag=False)

    datasetRefs = registry.queryDatasets('postISRCCD', dataId=dataId, collections  = collections)
    for i, ref in enumerate(datasetRefs):
        postisrccd  = butler.get(ref)
    mu_pisr,med_pisr,std_pisr,sigMad_pisr,vmin_pisr,vmax_pisr = ComputeStatfromImage(postisrccd.getImage().array,remove_zero_flag=False)

    mask = calexp.mask.array
    the_mask = np.where(mask==0,1,0)

    mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,vmin_ce_m,vmax_ce_m = ComputeStatfromImage(calexp.getImage().array*the_mask ,remove_zero_flag=True)
    mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,vmin_bkg_m,vmax_bkg_m = ComputeStatfromImage(bkgd.getImage().array*the_mask,remove_zero_flag=True)
    mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m,vmin_pisr_m,vmax_pisr_m = ComputeStatfromImage(postisrccd.getImage().array*the_mask,remove_zero_flag=True)

    del calexp,bkgd,postisrccd,mask,the_mask

    df.loc[index] = [index,visitId,ccdvisitId,band, det,
                     zeroPoint,airmass,skyBg,skyNoise,expTime,
                     expo_photocalibconstant_mean,expo_photocalibconstant_error,
                     magzero,magzero_rms,magzero_nobj,
                     bgmean,bgvar,
                     mu_ce,med_ce,std_ce,sigMad_ce,
                     mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,
                     mu_bkg,med_bkg,std_bkg,sigMad_bkg,
                     mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,
                     mu_pisr,med_pisr,std_pisr,sigMad_pisr,
                     mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m
                    ]
   


    if index<10 or index%100 == 0:
        print(f"{index}) ======================{visitId} == {band}  == {det} =====================")
        print('* photocalib ::')
        print(photocalib)
        print(expo_photocalibconstant_mean,convert_fluxtomag(expo_photocalibconstant_mean) )
        print("magzero,magzero_rms,magzero_nobj :",magzero,magzero_rms,magzero_nobj)
        print("bgmean,bgvar :",bgmean,bgvar)
        print("* calexp ::")
        print(mu_ce,med_ce,std_ce,sigMad_ce,vmin_ce,vmax_ce)
        print(mu_ce_m,med_ce_m,std_ce_m,sigMad_ce_m,vmin_ce_m,vmax_ce_m)
        print("* calexpBackground :: ")
        print(mu_bkg,med_bkg,std_bkg,sigMad_bkg,vmin_bkg,vmax_bkg)
        print(mu_bkg_m,med_bkg_m,std_bkg_m,sigMad_bkg_m,vmin_bkg_m,vmax_bkg_m)
        print("* postisrccd :: ")
        print(mu_pisr,med_pisr,std_pisr,sigMad_pisr,vmin_pisr,vmax_pisr)
        print(mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m,vmin_pisr_m,vmax_pisr_m) 


        

    if NMAX>0 and index>NMAX :
        break
   

0) ======================2023051100262 == r  == 0 =====================
* photocalib ::
spatially constant with mean: 30.4905 error: 0.0514512
30.49046409792168 27.689589912878567
magzero,magzero_rms,magzero_nobj : 23.9967867760794 0.00183212641679645 36
bgmean,bgvar : 11.3790014454661 0.0632380885261264
* calexp ::
1.7696625 0.11838882 254.89998 7.989821800231933 -23.851076583981513 24.087854217410086
0.00023732598019618814 0.007767405128106475 7.9552807771891105 7.9355474649099165 -23.798874989601643 23.814409799857856
* calexpBackground :: 
11.279222 11.335263 0.25390127 0.2340415288925171 10.63313866558075 12.037387838935851
11.287554528754608 11.33798885345459 0.24836555978148378 0.2255848880767822 10.661234189224244 12.014743517684936
* postisrccd :: 
13.167816 11.390819 266.133 7.9942487760543814 -12.591927732276915 35.373564924049376
11.287792617046525 11.287638664245605 7.959123604637137 7.940485975456237 -12.533819262123107 35.109096590614314
1) ======================20230511

In [39]:
df

,idx,visitId,ccdVisitId,band,detector,photocalib_m,photocalib_e,magzero,magzero_rms,magzero_nobj,...,std_bkg_m,sigMad_bkg_m,mu_pisr,med_pisr,std_pisr,sigMad_pisr,mu_pisr_m,med_pisr_m,std_pisr_m,sigMad_pisr_m
0,0,2023051100262,40919696896,r,0,30.490464,0.051451,23.996787,0.001832,36,...,0.248366,0.225585,13.167816,11.390819,266.132996,7.994249,11.287793,11.287639,7.959124,7.940486
1,1,2023051100264,40919697408,i,0,25.507239,0.042118,24.190538,0.001793,57,...,0.179049,0.156368,38.809303,28.262280,1188.283691,8.728232,28.132761,28.130495,8.683864,8.655220
2,2,2023051100268,40919698432,r,0,27.956394,0.037252,24.090994,0.001447,15,...,0.209981,0.178780,12.794616,10.863235,288.811523,7.990491,10.761060,10.754449,7.955056,7.935189
3,3,2023051100270,40919698944,i,0,26.273077,0.039450,24.158420,0.001630,32,...,0.153219,0.113119,22.809006,24.114464,353.772095,8.576545,24.015178,23.997007,8.539144,8.510426
4,4,2023051100271,40919699200,i,0,26.568090,0.053103,24.146296,0.002170,36,...,0.192677,0.186060,25.825996,23.512068,298.147797,8.556888,23.397298,23.379817,8.509870,8.486023
5,5,2023051100273,40919699712,r,0,28.065410,0.033992,24.086768,0.001315,49,...,0.087576,0.081857,11.863583,10.161476,322.370056,7.893755,10.070424,10.065777,7.864033,7.844471
6,6,2023051100274,40919699968,r,0,28.351260,0.055739,24.075766,0.002135,35,...,0.210382,0.168272,12.169188,10.144657,293.685516,7.943615,10.064845,10.054284,7.908459,7.892278
7,7,2023051100276,40919700480,i,0,28.983574,0.052112,24.051817,0.001952,40,...,0.163880,0.190237,23.024500,22.981083,120.547798,8.513059,22.874741,22.861481,8.476606,8.450306
8,8,2023051100277,40919700736,i,0,27.473770,0.045506,24.109901,0.001798,42,...,0.126935,0.102888,25.492258,23.214010,291.218994,8.545373,23.098445,23.084429,8.506419,8.475661
9,9,2023051100278,40919700992,g,0,23.926036,0.042526,24.260020,0.001930,26,...,0.112363,0.104779,5.730689,3.838008,300.974121,7.537274,3.753979,3.765186,7.552335,7.500762


In [35]:
if NMAX>0:
    outputfile = fn_ccdVisit_tracts_patches.split(".")[0]+f"_calexpinfo_{NMAX}.csv"
else:
    outputfile = fn_ccdVisit_tracts_patches.split(".")[0]+f"_calexpinfo.csv"
print(f"output file : {outputfile}")

output file : ccdVisittractpatch_LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240311_w_2024_10_PREOPS-4985_calexpinfo.csv


In [40]:
df.to_csv(outputfile)